In [1]:
# Importing the necessary libraries
import os  # For working with files and directories
import torch  # PyTorch
import torch.nn as nn  # Neural network module
import torchvision.transforms as transforms  # image transformation
from torchvision.models import resnet50, ResNet50_Weights  # Importing the ResNet50 model
from PIL import Image  # image processing library
import psycopg2  # PostgreSQL database adapter
import logging # Logging module

In [2]:
# TODO list

# API
# Error handling
# Logging
# Docker
# Vector Index


In [ ]:
# ENVIRONMENT VARIABLES

## Model: resnet50

## Dimension: 2048

## pgsql connection

## Top_k

In [2]:
def load_model():
    """
    Loads the pre-trained ResNet50 model and sets it to evaluation mode.

    Returns:
        model: The loaded ResNet50 model.
    """
    # Use the recommended weights argument instead of pretrained
    weights = ResNet50_Weights.IMAGENET1K_V1  # Alternatively, use ResNet50_Weights.DEFAULT for the latest weights
    model = resnet50(weights=weights)  # Load the model with specified weights
    
    # Remove the last fully connected layer to get 2048-dimensional outputs
    model = torch.nn.Sequential(*list(model.children())[:-1])  # Keep all layers except the last one
    model.eval()  # Set the model to evaluation mode
    return model

In [3]:
def preprocess_image(image_path):
    """
    Preprocess the image.

    parameters:
        image_path (str): Image File Path

    Returns:
        torch.Tensor: Preprocessed image tensor
    """
    # Defining Image Transformation Operations
    transform = transforms.Compose([
        transforms.Resize(256),  # Resizing images
        transforms.CenterCrop(224),  # Center Cropped Image
        transforms.ToTensor(),  # Converting images to tensors
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # standardization
    ])
    
    image = Image.open(image_path).convert("RGB")  # Read images and convert to RGB format
    image_tensor = transform(image).unsqueeze(0)  # Add Batch Dimension
    return image_tensor

In [4]:
def generate_image_vector(model, image_path):
    """
    Generate a 2048-dimensional vector of images.

    parameters:
        model: Pre-trained ResNet50 model
        image_path (str): Image File Path

    Returns:
        numpy.ndarray: 2048-dimensional image vector
    """
    image_tensor = preprocess_image(image_path)  # Preprocessed images
    with torch.no_grad():  # Disable gradient calculation
        vector = model(image_tensor).numpy()  # Generate vectors using models and convert to numpy arrays
    return vector.flatten()  # Flatten vectors for easy storage

In [5]:
# Add error handling: database connection error, table not found, item duplicate, etc.

# Configure logging
logging.basicConfig(filename='image_vector_insertion.log', level=logging.ERROR,
                    format='%(asctime)s - %(levelname)s - %(message)s')

def insert_vector_to_db(connection, image_name, vector):
    """
    Inserts image vectors into a PostgreSQL database.

    parameters:
        connection: PostgreSQL Database Connection
        image_name (str): Image file name (no extension)
        vector (numpy.ndarray): 2048-dimensional image vector
    
    Returns:
        bool: True if the insertion is successful, False if there's a duplicate key error
    """

    try:
        cursor = connection.cursor()  # Create database cursor
        # Perform an insertion operation
        cursor.execute(
            "INSERT INTO image_info (image_name, image_path, vector) VALUES (%s, %s, %s)",
            (image_name, None, vector.tolist())  # image_path temporarily empty
        )
        connection.commit()  # Commit the transaction
        cursor.close()  # Close cursor
        return True  # Indicate successful insertion
    
    except psycopg2.errors.UniqueViolation:  # Catch the unique constraint violation error
        connection.rollback()  # Rollback the transaction
        logging.error(f"Duplicate entry for image_name: {image_name}. Skipping insertion.")
        print(f"Duplicate entry for image '{image_name}'. Skipping insertion.")
        return False  # Indicate failure due to duplicate key error
    
    except Exception as e:  # Catch any other database-related errors
        connection.rollback()  # Rollback the transaction
        logging.error(f"Error inserting image {image_name}: {str(e)}")
        print(f"Error inserting image '{image_name}': {str(e)}")
        return False  # Indicate failure

In [6]:
def count_subdirectories_and_images(image_dir):
    """
    Count the total number of subdirectories and images in the specified directory.

    Parameters:
        image_dir (str): Root directory of the image files

    Returns:
        total_subdirs (int): Total number of subdirectories
        total_images (int): Total number of images
    """
    total_subdirs = 0
    total_images = 0

    for root, dirs, files in os.walk(image_dir):
        # Count subdirectories
        total_subdirs += len(dirs)
        # Count image files (only .jpg, .jpeg, .png)
        total_images += len([file for file in files if file.endswith(('.jpg', '.jpeg', '.png'))])

    # print(f"Total number of subdirectories: {total_subdirs}")
    # print(f"Total number of images: {total_images}")
    
    return total_subdirs, total_images

# Example usage
# image_dir = '/Users/Tommy/AI/image-search/image-search-pgvector/image-test'
# subdirs, images = count_subdirectories_and_images(image_dir)
# print(f"Total number of subdirectories: {subdirs}")
# print(f"Total number of images: {images}")

In [7]:
def process_images_in_directory(image_dir, model, connection):
    """
    Recursively read all the images in the catalog and insert their vectors into the database.

    Parameters:
        image_dir (str): Root directory of the image files
        model: Pre-trained ResNet50 model
        connection: PostgreSQL database connection
    """
    for root, _, files in os.walk(image_dir):
        # Filter the files to include only images
        image_files = [file for file in files if file.endswith(('.jpg', '.jpeg', '.png'))]
        
        for file in image_files:
            image_path = os.path.join(root, file)  # Get the full image path
            print(f"Processing {image_path}...")  # Print the path of the image being processed

            vector = generate_image_vector(model, image_path)  # Generate image vectors
            
            image_name = os.path.splitext(file)[0]  # Use the file name (without extension) as the ID
            
            # Write to the database and check if insertion succeeded
            success = insert_vector_to_db(connection, image_name, vector)
            
            if not success:  # Skip to the next image if insertion failed due to duplication
                continue

In [8]:
# main embeding functio

def main(image_dir):
    """
    main function that executes the entire process.
    
    parameters:
        image_dir (str): Root directory of the image file
    """
    # TODO: Add error handling: database connection error
    # Connecting to a PostgreSQL Database
    connection = psycopg2.connect(
        dbname='imageSearch',  # database name
        user='postgres',  # user ID
        password='test-postgres',  # cryptographic
        host='localhost',  # RDSTerminal node of the instance
        port='5432'  # PostgreSQL ports
    )

    model = load_model()  # Loading Models
    
    # count_subdirectories_and_images(image_dir)  # Count the number of subdirectories and images
    subdirs, images = count_subdirectories_and_images(image_dir)
    print(f"Total number of subdirectories: {subdirs}")
    print(f"Total number of images: {images}")

    process_images_in_directory(image_dir, model, connection)  # process image
    
    connection.close()  # Close the database connection

In [ ]:
# Execute the main function with the image directory as an argument
main('/Users/Tommy/AI/image-search/clothing-images')  # Image directory path

# Query the Database for Similar Vectors

## Search
1. the nearest neighbors by L2 distance(<->), measures the "straight line" distance between two vectors in space.
2. inner product (<#>), be used to compare the projection of one vector on another.
3. cosine distance (<=>), measures the cosine of the angle between two vectors. It’s widely used in machine learning to find similarities between two data points.
4. L1 distance (<+>, added in 0.7.0)
Note: <#> returns the negative inner product since Postgres only supports ASC order index scans on operators.

## Index
1. HNSW: more better, but vector - up to 2,000 dimensions, we use 2048 dimensions.
2. IVFFlat

In [5]:
def search_similar_images(connection, input_vector, top_k):
    """
    Search for similar images in the database using pgvector.

    Args:
        connection: PostgreSQL database connection.
        input_vector (list): The 2048-dimensional input vector.
        top_k (int): The number of most similar images to retrieve.

    Returns:
        list: A list of tuples containing image_names and distances.
    """
    # print(input_vector) 
    # [ 8.50246012e-01 -5.55582345e-01 -1.58667660e+00 -4.00121570e-01 ]
    
    vector_str = ','.join(map(str, input_vector))  # Create a string of comma-separated values
    
    # print(vector_str)
    vector_str = f'[{vector_str}]'  # Format the string as a list

    print("image_vector: ", vector_str)
    # 0.850246012, -0.555582345, -1.5866766, -0.40012157

    # cosine distance (<=>)
    query = """
    SELECT product_code, image_name, vector <=> %s AS distance
    FROM image_info
    ORDER BY distance
    LIMIT %s;
    """
    
    with connection.cursor() as cursor:
        cursor.execute(query,(vector_str,top_k))  # Execute the query
        results = cursor.fetchall()
    
    return results

In [6]:
def find_similar_images(image_path, model, connection, top_k):
    """
    Find the most similar images to the input image from the database.

    Args:
        image_path (str): Path to the input image.
        model: Pre-trained ResNet50 model.
        connection: PostgreSQL database connection.
        top_k (int): Number of similar images to retrieve.

    Returns:
        list: List of similar image names and distances.
    """
    # Step 1: Convert input image to vector
    input_vector = generate_image_vector(model, image_path)  # Generate image vectors

    # Step 2: Search for similar images in the database
    similar_images = search_similar_images(connection, input_vector, top_k)
    
    return similar_images

In [ ]:
# main search function

TOP_K = 6 # Number of similar images to retrieve

image_path = "/Users/Tommy/AI/image-search/image-search-pgvector/image-test/088/0880060002.jpg"

# Connecting to a PostgreSQL Database
connection = psycopg2.connect(
        dbname='imageSearch',  # database name
        user='postgres',  # user ID
        password='test-postgres',  # cryptographic
        host='localhost',  # RDSTerminal node of the instance
        port='5432'  # PostgreSQL ports
    )

model = load_model()  # Load the ResNet50 model

similar_images = find_similar_images(image_path, model, connection, TOP_K)

print(f"Top {TOP_K} similar images: {similar_images}")

# for image_name, distance in similar_images:
#     print(f"Image: {image_name}, Distance: {distance}")

connection.close()  # Close the database connection
